In [6]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import boxcox
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, SVR
from precision_recall_cutoff import precision_recall_cutoff

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [7]:
# changing sales and salary dummies
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis =  1 ), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [8]:
# engineering interactions
turnover['interaction_1'] = np.where((turnover['satisfaction_level'] >= .115) & (turnover['satisfaction_level'] <= .465) & (turnover['number_project'] > 2.5), 1, 0)
turnover['interaction_2'] = np.where((turnover['satisfaction_level'] <= .465) & (turnover['number_project'] <= 2.5) & (turnover['last_evaluation'] <= .575), 1, 0)
turnover['interaction_3'] = np.where((turnover['satisfaction_level'] > .465) & (turnover['time_spend_company'] <= 4.5) & (turnover['average_montly_hours'] <= 290.5), 1, 0)

In [9]:
# defining imput and target
x = turnover[['interaction_3', 'interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
y = turnover['left']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [22]:
Ada_param_grid = {'n_estimators': [100, 300],
                    'base_estimator__min_samples_split': [10, 15],
                    'base_estimator__min_samples_leaf': [5, 7],
                    'base_estimator__max_depth': [3, 5, 7],
                    'learning_rate': [0.01]}

# running grid search 
ada_grid = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), 
                        param_grid = Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(x_train, y_train)

# extracting best hyper params
ada_grid.best_params_

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` wa

{'base_estimator__max_depth': 5,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__min_samples_split': 15,
 'learning_rate': 0.01,
 'n_estimators': 300}

In [25]:
# building random forest
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 5, max_depth = 5),
                            n_estimators = 300, learning_rate = 0.01).fit(x_train, y_train)

# predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

# likilyhookd to label
ada_labels = precision_recall_cutoff(y_test, ada_pred)

print(classification_report(y_test, ada_labels))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2286
           1       0.94      0.92      0.93       714

    accuracy                           0.97      3000
   macro avg       0.96      0.95      0.95      3000
weighted avg       0.97      0.97      0.97      3000



In [23]:
# running grid search 
ada_grid2 = RandomizedSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), 
                        param_distributions = Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1, n_iter = 10).fit(x_train, y_train)

# extracting best hyper params
ada_grid2.best_params_

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` wa

{'n_estimators': 300,
 'learning_rate': 0.01,
 'base_estimator__min_samples_split': 10,
 'base_estimator__min_samples_leaf': 7,
 'base_estimator__max_depth': 5}

In [26]:
# building random forest
ada_md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 7, max_depth = 5),
                             n_estimators = 300, learning_rate = 0.01).fit(x_train, y_train)

# predicting on test
ada_pred2 = ada_md2.predict_proba(x_test)[:,1]

# likilyhookd to label
ada_labels2 = precision_recall_cutoff(y_test, ada_pred2)

print(classification_report(y_test, ada_labels2))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2286
           1       0.94      0.92      0.93       714

    accuracy                           0.97      3000
   macro avg       0.96      0.95      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [27]:
# based on my results randomized search offers a slight increase in performace so i would use that model to predict left